In [1]:
#!conda install -c conda-forge fbprophet -y


In [2]:
 #conda update -n base -c defaults conda

In [5]:
#!pip install --upgrade plotly
!pip install pystan==2.19.1.1 prophet

  Using cached https://files.pythonhosted.org/packages/f0/fa/c382f0ac5abe9f0a4df9d874a5e8843db035fe2f071b5c00a545b1e3c10b/prophet-1.0.1.tar.gz
  Running setup.py clean for prophet
Failed to build prophet
  Running setup.py install for prophet: started
    Running setup.py install for prophet: finished with status 'error'


  ERROR: Complete output from command 'C:\Users\katay\Anaconda3\python.exe' -u -c 'import setuptools, tokenize;__file__='"'"'C:\\Users\\katay\\AppData\\Local\\Temp\\pip-install-lim5jn8_\\prophet\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\katay\AppData\Local\Temp\pip-wheel-xtyk1iwf' --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib
  creating build\lib\prophet
  creating build\lib\prophet\stan_model
  C:\Users\katay\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
    warnings.warn(msg, UserWarning)
  INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f5236004a3fd5b8429270d00efcc0cf9 NOW.
  error: Microsoft Visual C++ 14.0 is required. Get it

In [6]:
from prophet import Prophet

ModuleNotFoundError: No module named 'prophet'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from  scipy import stats
!pip install prophet
url='https://drive.google.com/file/d/1IHLUqlQuSVMiKjnEb3EVTkzIjF413-Bl/view?usp=sharing'
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
data = pd.read_csv(url2)
data.head()


In [ ]:
categorical=[col for col in data.columns if data[col].dtype=="object"]
for i,j in enumerate(categorical):
    print("{0}: {1} has the unique elements:{2}".format(i+1,j,data[j].unique()))

In [ ]:
#to check missing values
data.isnull().any() 

In [ ]:
data.drop(['Column1', 'Column2','Column3', 'Column4'], axis = 1, inplace = True)

In [ ]:
# check each field in the data
data.describe()

#### <span style="color:#bf616a">Note:</span>
#### <span style="color:#5e81ac">What is a reaction? num_reactions = SUM of (num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys)</span>


In [ ]:
# group by type, get mean and max value of num_shares.
shares_per_type = data.groupby('status_type').agg({'num_shares': ['mean', 'max']})
print(shares_per_type)


#### <span style="color:#5e81ac">=================Data visiualization to get a better idea of the data==============</span>

In [ ]:
#num of reactions by status type

reactions_percentage=data.groupby("status_type")["num_reactions"].count()
colors = ["#babeee","#7573b6","#ecf4be","#90d2c3"]
fig,ax=plt.subplots(figsize=(15,8))
reactions_percentage.plot.pie(ax=ax,colors=colors,autopct="%0.1f%%",fontsize=20)

In [ ]:
sns.set()
fig,ax=plt.subplots(figsize=(10,8))
test=data.groupby("status_type")["num_shares"].max()
test=test/sum(test)*100
color=["#58508d","#bc5090","#ff6361","#ffa600"]
ax.bar(test.index,test.values,color=color,alpha=0.8)
ax.set_xlabel("Status Type",fontsize=15)
ax.set_ylabel("Shares",fontsize=15)
ax.set_title("Distribution of Shares and Types",fontsize=18)


#### <span style="color:#bf616a">Note:</span>
#### <span style="color:#5e81ac">Photo and Video have more shares.</span>

In [ ]:
cor_matrix = data.corr()
round(cor_matrix,2)

In [ ]:
sns.heatmap(cor_matrix);

#### <span style="color:#bf616a">Note:</span>
#### <span style="color:#5e81ac">There is high correlation between number of reactions (num_reactions) and number of likes (num_likes) - around 99%</span>
#### <span style="color:#5e81ac"> Pretty high correlation between number of loves (num_loves) and number of shares (num_shares) - around 0.82%</span>
#### <span style="color:#5e81ac">Another high correlation between number of shares (num_shares) and number of comments (num_comments) - around 0.64%</span>


In [ ]:
#print(data);

In [ ]:
data["Date_time"] = data["status_published"].astype("datetime64[ns]")
#data['MM-DD'] = data['Date_time'].dt.strftime('%m-%d');
data["day_time"] = pd.cut(pd.to_datetime(data.Date_time).dt.hour,
bins=[0, 6, 12, 18, 24],
labels=["night", "morning", "afternoo", "evening"],
right=False,
include_lowest=True)


In [ ]:
print(data)

In [ ]:
sns.set()
fig,ax=plt.subplots(figsize=(10,8))
dayplot=data.groupby("day_time")['num_reactions'].count()
dayplot=dayplot/sum(dayplot)*100
color=["#a3be8c","#b48ead","#bf616a","#ebcb8b"]
ax.bar(dayplot.index,dayplot.values,color=color,alpha=0.7)
for i in ax.patches:
    ax.text(i.get_x()+0.25,i.get_height()+0.2,str(round(i.get_height(),2))+"%")
ax.set_xlabel("Times of Day")
ax.set_ylabel("percentage")
ax.set_title("Distribution of Reactions during different parts of the day",fontsize=15)


####  <span style="color:#bf616a">Suggestion:</span> <span style="color:#5e81ac">The historical data shows more reactions in the Morning and Night, would be a good idea to post more in the morning and night.</span>
####  <span style="color:#bf616a">Future work:</span> <span style="color:#5e81ac">Try to brakdown by hour / weekdays / weekend / Month to find the best time to post or boost.</span>

In [ ]:
# group by day_time, get mean and max value of num_reactions.
reaction_per_dat_time = data.groupby('day_time').agg({'num_reactions': ['mean', 'max']})
print(reaction_per_dat_time)

In [ ]:
len(data)

In [ ]:
df_train=data[0:500]
df_test=data[500:1000]

In [ ]:
len(df_train)

In [ ]:
len(df_test)

In [ ]:
from prophet import Prophet
m = Prophet()
m.fit(df_train)